### Step Two - Blend Data
#### Activity Three - Determine chain ownership
#### Activity Four - Determine hospital affiliation
#### Activity Five - Determine geograohical region & division
#### Activity Six - Determine urbanicity
#### Activity Seven - Determine years in operation
#### Activity Eight - Final Clean-up

Data sources:

* Dialysis Facility Report 2018
* CMS Manual - Facility Type Based on CCN Range Table
* US Census State Region Division Table
* Rural Urban Area Code

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 60)

#### Activity Three -  Determine chain ownership

In [ ]:
dtype_dict= {'CCN': str,    
             'Network': str, 
             'ZipCode': str}

dfr=pd.read_csv("../input/activity-two-blend-acs-data/InterimDataset2.csv", parse_dates=True, dtype=dtype_dict)
print("\nThe DFR data frame has {0} rows or facilities and {1} variables or columns\n".format(dfr.shape[0], dfr.shape[1]))

In [ ]:
dfr["ChainOwner"].describe()

The count of chain owners 5794 is less than the totla count of facilities 6574. This indicates there are facilities with null value in chain owner. Let's find out how many facilities don't have a chain owner. These are considered independent facilities.

In [ ]:
len(dfr[dfr['ChainOwner'].isnull()])  

This is due to the fact that there are 780 independent facilities. Transform this column into the following four levels:
* DaVita
* FMC
* Other Chains
* Independent

In [ ]:
chain_dict = {'DAVITA': 'DaVita',
              'FRESENIUS MEDICAL CARE': 'FMC'}      

def get_owner(owner):
    if owner is np.NaN:
        return 'Independent'
    elif owner in chain_dict.keys():
        return chain_dict[owner]
    else:
        return "Other Chains"

dfr["ChainOwner"] = dfr["ChainOwner"].apply(get_owner)      
dfr['ChainOwner'].value_counts()

In [ ]:
dfr['ChainOwner'].value_counts().sort_index() # sort by index. This is the default. so we can omit .sort_index()

In [ ]:
dfr['ChainOwner'].value_counts().sort_values() # sort by value

DaVita has the most number of facilities 2,384 followed by FMC 2,285

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Numbere of Facilities by Chain Owner")
dfr['ChainOwner'].value_counts().plot(kind='bar')   # This is Pandas plot, not as fancy as Seaborn

In [ ]:
plt.figure(figsize=(10,6))
# plt.ylabel("Number of Facilities")     # did not work
plt.xticks(rotation=-45)                                 # control the display of the categories
sort_order = dfr['ChainOwner'].value_counts().index
ax = sns.countplot(x='ChainOwner', data=dfr, order=sort_order)   # This is Seaborn plot
ax.set(ylabel='Number of Facilities', xlabel="Dialysis Chain Owner")

#### Activity Four - Determine Hospital Affiliation

### 2779J - ESRD CMS Certification Numbers
(Rev. 29, Issued: 04-20-07; Effective/Implementation Dates: 10-01-2007)

It is important for both reimbursement and survey purposes to assign the ESRD facility
the correct CCN in accordance with the guidelines contained in §2779.A.1.

ESRD facilities and their CCN are as follows:
* 0001-0879 Short Term (General and Specialty) Hospitals
* 2000-2299 Long Term Hospitals
* 2300-2499 Chronic Renal Disease Facilities (Hospital-Based)
* 2500-2899 Non-Hospital Renal Disease Treatment Centers
* 2900-2999 Independent Special Purpose Renal Disease Facilities
* 3300-3399 Children’s Hospitals
* 3500-3699 Renal Disease Treatment Centers (Hospital Satellites)
* 3700-3799 Hospital-Based Special Purpose Renal Dialysis Facilities

Source: https://www.cms.gov/Regulations-and-Guidance/Guidance/Manuals/Downloads/som107c02.pdf

In [ ]:
dict2 = {
(1,879): 'Short Term Hospitals',
(2000,2299): 'Long Term Hospitals',
(2300,2499): 'Hospital-Based Facilities',
(2500,2899): 'Non-Hospital Facilities',
(2900,2999): 'Non-Hospital Facilities (Special Purpose)',
(3300,3399): 'Children’s Hospitals',
(3500,3699): 'Hospital Satellites Facilities',
(3700,3799): 'Hospital-Based Facilities (Special Purpose)'
       }

dict = {
(1,879): 'Hospitals',
(2000,2299): 'Hospitals',
(2300,2499): 'Hospital-Based Facilities',
(2500,2899): 'Non-Hospital Facilities',
(2900,2999): 'Non-Hospital Facilities',
(3300,3399): 'Hospitals',
(3500,3699): 'Hospital Satellites Facilities',
(3700,3799): 'Hospital-Based Facilities'
       }

def get_facility_type(ccn):
    last_four = int(ccn[2:6])
    fac_type = 'Unknown'
    for key in dict: 
        lower_bound, upper_bound = key # unpacking the lower and upper bounds in the the range
        if lower_bound <= last_four and last_four <= upper_bound:
            fac_type = dict[key]
            break
        else: 
            next
    return fac_type

dfr["HospitalAffiliation"] = dfr['CCN'].apply(get_facility_type)    # how elegant
dfr["HospitalAffiliation"].value_counts()

In [ ]:
dfr[dfr['HospitalAffiliation'].isnull()]

In [ ]:
plt.figure(figsize=(10,6))
# plt.ylabel("Number of Facilities")     # did not work
plt.xticks(rotation=-45)                                 # control the display of the categories
sort_order = dfr['HospitalAffiliation'].value_counts().index
ax = sns.countplot(x='HospitalAffiliation', data=dfr, order=sort_order)   # This is Seaborn plot
ax.set(ylabel='Number of Facilities', xlabel="Hospital Affiliation")

#### Activity Five - Determine geograohical region & division

In [ ]:
dfr["StateCode"].describe()

There are 56 state codes. 50 states, 1 District of Columbia, and 5 territories

In [ ]:
dfr['StateCode'].unique()

In [ ]:
column_dict ={'State Code':'StateCode'}  # change the column name to match with DFR dataset
region = pd.read_csv("../input/rmudsc/Census-Bureau-Regions.csv")
region.rename(columns=column_dict, inplace=True)
region.info()

51 state codes. 50 states, 1 District ofColumbia. It does not include the 5 territories

In [ ]:
region.head()

In [ ]:
region.Region.value_counts()

In [ ]:
plt.figure(figsize=(10,6))
region.Region.value_counts().plot(kind="bar")

The DFR dataset has more state codes. Find out the differences. It turns out the Census dataset only has 50 states and the District of Columbia. The DFR data set includes 50 states + DC + the five oversea territories: 

* AS = American Samoa 
* GU = Guan 
* MP = Northern Mariana Islands 
* PR = Perto Rico 
* VI = US Virgin Islands

In [ ]:
set(dfr['StateCode'].unique()) - set(region['StateCode'].unique())

Merge the two datasets based on state code.

In [ ]:
dfr = pd.merge(dfr, region, on='StateCode', how='left')
dfr.Region.unique()

In [ ]:
dfr[dfr["Region"].isnull()].sample(3)

In [ ]:
dfr['Region'].fillna('Oversea', inplace=True)
dfr.Region.unique()

In [ ]:
dfr['Division'].unique()

In [ ]:
# replace the null value with "Oversea" to represent the oversea territories
dfr['Division'].fillna('Oversea', inplace=True)
dfr.Division.unique()

#### Activity Six - Determine urbanicity

In [ ]:
dtype_dict={'ZIPCODEN':str}
column_dict={'ZIPCODEN': 'ZipCode',
            'RUCA30': 'RUCA'}
rural=pd.read_csv("../input/rmudsc/URCA-310.csv", dtype=dtype_dict, usecols=column_dict.keys())
rural.rename(columns=column_dict, inplace=True)
rural.info()

In [ ]:
dfr = pd.merge(dfr, rural, on='ZipCode', how='left')
dfr.shape

In [ ]:
def get_urban(ruca):
    if ruca < 4:      # 1.x, 2.x, and 3.x
        return 'Metropolitan'
    elif ruca < 7:    # 4.x, 5.x, and 6.x
        return 'Micropolitan'   
    elif ruca < 10:   # 7.x, 8.x, and 9.x
        return "Small Town"
    elif ruca < 11:   # 10.x
        return "Rural Area"
    else:             # NaN
        return 'Unknown'
# 1,2,and 3 are considered urban, 4 to 10 are considered rural.    

dfr['Urbanicity'] = dfr['RUCA'].apply(get_urban)

In [ ]:
#dfr[dfr['Urbanicity'] == 'Unknown']
dfr['Urbanicity'].unique()

In [ ]:
dfr.Urbanicity.value_counts().plot(kind="bar")

In [ ]:
def get_urban(ruca):
    if ruca < 4:      # 1.x, 2.x, and 3.x
        return 'Urban'
    elif ruca < 11:   # 4 to 10
        return "Rural"
    else:             # NaN
        return 'Unknown'
# 1,2,and 3 are considered urban, 4 to 10 are considered rural.    

dfr['Urbanicity2'] = dfr['RUCA'].apply(get_urban)

In [ ]:
dfr.Urbanicity2.value_counts().plot(kind="bar")

#### Activity Seven - Determine years in operation

In [ ]:
dfr['InitCertDate2'] =  pd.to_datetime(dfr['InitCertDate'], format='%d%b%Y')
dfr[['InitCertDate', 'InitCertDate2']].sample(5)

In [ ]:
dec_31_2016 = pd.to_datetime('2016-12-31')
dfr['YearsInOps'] =  ((dec_31_2016 - dfr['InitCertDate2'])/np.timedelta64(1, 'Y'))
#dfr['Months In Ops'] =  ((dec_31_2016 - dfr['Init. Cert. Date 2'])/np.timedelta64(1, 'M'))
dfr[dfr['YearsInOps'].isnull() == True]

#### Activity Eight - Final clean-up

In [ ]:
#  Rreview all columns 
dfr.columns

In [ ]:
# Drop the unused ones
drop_columns = ["Unnamed: 0", 
                "RUCA",
                'InitCertDate',
                'InitCertDate2']
                
dfr.drop(drop_columns, axis=1, inplace=True)  
dfr.columns

In [ ]:
# Review data elements from five random facilities 
dfr.sample(5)

In [ ]:
# DialyzerReuse has missing values
# Python built-in number NaN is Not-a-Numberrepresenting missing values)
dfr['DialyzerReuse'].dtype

In [ ]:
dfr['DialyzerReuse'].describe()

In [ ]:
# Find out how many facilities do not reuse dialyzer
len(dfr[dfr['DialyzerReuse'].isnull()])  

In [ ]:
# Convert the data from float to string
dfr['DialyzerReuse'] = dfr['DialyzerReuse'].astype(str)
dfr['DialyzerReuse'].describe()

In [ ]:
def convert_dialyzer_reuse(dialyzer_reuse):
    if dialyzer_reuse == '1.0':    
        return 'Dialyzer Reuse'
    else:             # NaN
        return 'No Dialyzer Reuse'
dfr['DialyzerReuse'] = dfr['DialyzerReuse'].apply(convert_dialyzer_reuse)
dfr['DialyzerReuse'].describe()

In [ ]:
## EveningShift has missing values
# Python built-in number NaN is Not-a-Numberrepresenting missing values)
dfr['EveningShift'].describe()

In [ ]:
# Find out how many facilities do not offer evening shift
len(dfr[dfr['EveningShift'].isnull()])  

In [ ]:
# Convert the data from float to string
dfr['EveningShift'] = dfr['EveningShift'].astype(str)
dfr['EveningShift'].describe()

In [ ]:
def convert_evening_shift(evening_shift):
    if evening_shift == '1.0':    
        return 'Evening Shift'
    else:             # NaN
        return 'No Evening Shift'
dfr['EveningShift'] = dfr['EveningShift'].apply(convert_evening_shift)
dfr['EveningShift'].describe()

Convert the compliance status from numeric to categorical.
Compliance Status
* 0 - met requirements,
* 1 - did not meet requirements but had an acceptable plan of correction,
* 2 - did not meet requirements 
* null - unknown

In [ ]:
dfr['ComplianceStatus'].dtype

In [ ]:
dfr['ComplianceStatus'] = dfr['ComplianceStatus'].astype(str)
dfr['ComplianceStatus'].describe()

In [ ]:
dfr['ComplianceStatus'].value_counts()

In [ ]:
def convert_compliance(status):
    if status == '0.0':     
        return 'Compliant'
    elif status == '1.0':   
        return "Not Compliant"
    elif status == '2.0':  
        return "Not Compliant"
    else:                          # NaN
        return 'Unknown'
dfr['ComplianceStatus'] = dfr['ComplianceStatus'].apply(convert_compliance)

In [ ]:
dfr.to_csv("DialysisCareQualityData.csv")

THE END of Activity Three to Seven.